In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('/home/prashant/incedo/test/Dataset2147b1d/train_file.csv')
test = pd.read_csv('/home/prashant/incedo/test/Dataset2147b1d/test_file.csv')

In [ ]:
data.head()

In [ ]:
len(data['Patient_ID'].unique())

In [ ]:
data.shape

In [ ]:
x = data['YEAR'].unique()
x.sort()
x

In [ ]:
data['LocationDesc'] = data['LocationDesc'].map(lambda x: x.lower())
test['LocationDesc'] = test['LocationDesc'].map(lambda x: x.lower())

In [ ]:
data['LocationDesc'].unique()

In [ ]:
# 1 <= drugs 
# 0 <= alcohol

data['Subtopic'].value_counts()

In [ ]:
data['Sex'].value_counts()

In [ ]:
data.info()

In [ ]:
cat_columns = ['Sex', 'Race', 'StratificationType', 'LocationDesc', 'YEAR', 'QuestionCode']
data[cat_columns] = data[cat_columns].astype('category')
data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)

test[cat_columns] = test[cat_columns].astype('category')
test[cat_columns] = test[cat_columns].apply(lambda x: x.cat.codes)

data

In [ ]:
data = data.drop(['Greater_Risk_Question', 'Description','GeoLocation'],axis=1)
test = test.drop(['Greater_Risk_Question', 'Description','GeoLocation'],axis=1)
data

In [ ]:
test

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression

y_train = data['Greater_Risk_Probability']
x_train = data.drop(['Greater_Risk_Probability'],axis =1)

X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,test_size =.1)

In [ ]:
# from sklearn.ensemble import RandomForestRegressor
# import matplotlib.pyplot as plt

# from subprocess import check_output
# from datetime import time
# import numpy as np
# # Establish model
# model = RandomForestRegressor(n_jobs=-1)

# estimators = np.arange(10, 200, 10)
# scores = []
# for n in estimators:
#     model.set_params(n_estimators=n)
#     model.fit(X_train, y_train_internal)
#     scores.append(model.score(X_test, y_test_internal))
# plt.title("Effect of n_estimators")
# plt.xlabel("n_estimator")
# plt.ylabel("score")
# plt.plot(estimators, scores)

params = {
'num_leaves': 15,
'max_bin': 100,
'min_data_in_leaf': 45,
'learning_rate': 0.01,
'min_sum_hessian_in_leaf': 0.000446,
'bagging_fraction': 0.55,
'bagging_freq': 5,
'max_depth': 14,
'save_binary': True,
'seed': 31452,
'feature_fraction_seed': 31415,
'feature_fraction': 0.51,
'bagging_seed': 31415,
'drop_seed': 31415,
'data_random_seed': 31415,
#'objective': 'multiclass',
'boosting_type': 'gbdt',
'verbose': 1,
'metric': 'rmse',
#'is_unbalance': False,
#     'categorical_feature': "name:branch_id"
# 'scale_pos_weight': len(train_df["target"] == 0) / sum(train_df["target"])
}

# Import the model we are using
#from sklearn.ensemble import RandomForestClassifier
# from tpot import TPOTClassifier
import lightgbm as lgb

d_train = lgb.Dataset(X_train, label=y_train_internal)
# Instantiate model with 1000 decision trees
#rf = RandomForestClassifier()

# Train the model on training data
#rm = rf.fit(X_train, y_train_internal)
rf = lgb.train(params, d_train,20000)

In [ ]:
pred_internal = rf.predict(test)
pred_internal = [round(pred) for pred in pred_internal]
from sklearn import metrics

In [ ]:
sub = pd.DataFrame({'Patient_ID': test['Patient_ID'], 'Greater_Risk_Probability': pred_internal})
sub.to_csv('tod_app_lgb.csv', index=False)
